In [1]:
import pandas as pd
import numpy as np
import json
import seaborn as sns 
import matplotlib.pyplot as plt
from TextRank import *

[nltk_data] Downloading package punkt to /home/harsh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/harsh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = pd.read_csv('metadata.csv', dtype={
    'pubmed_id': str,
    'Microsoft Academic Paper ID': str, 
    'doi': str
})

In [3]:
import glob
all_json = glob.glob('/home/harsh/CDAC/CORD-19-research-challenge/**/*.json', recursive=True)
len(all_json)

59311

In [4]:
all_json1=[]
for i in range(0,25000):
    all_json1.append(all_json[i])


In [5]:
len(all_json1)

25000

In [6]:
with open(all_json[0]) as file:
    first_entry = json.load(file)
    #print(json.dumps(first_entry, indent=4))

In [7]:
class FileReader:
    def __init__(self, file_path):
        with open(file_path) as file:
            content = json.load(file)
            self.paper_id = content['paper_id']
            self.abstract = []
            self.body_text = []
            for entry in content['abstract']:
                self.abstract.append(entry['text'])
            for entry in content['body_text']:
                self.body_text.append(entry['text'])
            self.abstract = '\n'.join(self.abstract)
            self.body_text = '\n'.join(self.body_text)
    def __repr__(self):
        return f'{self.paper_id}: {self.abstract[:200]}... {self.body_text[:200]}...'
first_row = FileReader(all_json[1])
print(first_row)

27fbfb693135818f95a81d461b54d2d3a359ab1e: Much evidence supports the notion that angiotensin II (Ang II), the central product of the renin-angiotensin system (RAS), may play a central role not only in the etiology of hypertension but also in ... Research on the renin-angiotensin system (RAS) began with the discovery of renin from the kidney by Tiegerstedt and Bergman in 1898. In 1940, a peptide that had vasoconstrictive effects in the RAS was...


In [8]:
dict_ = {'paper_id': [], 'abss': [], 'body_text': []}
for idx, entry in enumerate(all_json1):
    if idx % (len(all_json1) // 10) == 0:
        print(f'Processing index: {idx} of {len(all_json1)}')
    content = FileReader(entry)
    dict_['paper_id'].append(content.paper_id)
    dict_['abss'].append(content.abstract)
    dict_['body_text'].append(content.body_text)
df_covid = pd.DataFrame(dict_, columns=['paper_id', 'abss', 'body_text'])
df_covid.head()

Processing index: 0 of 25000
Processing index: 2500 of 25000
Processing index: 5000 of 25000
Processing index: 7500 of 25000
Processing index: 10000 of 25000
Processing index: 12500 of 25000
Processing index: 15000 of 25000
Processing index: 17500 of 25000
Processing index: 20000 of 25000
Processing index: 22500 of 25000


,paper_id,abss,body_text
0,b5d797140d4b3daf7b7ea753783daaad476e2eb0,Sequence characterization of the near full-len...,"T he sequences of two HIV-1 isolates, BCF-Diou..."
1,27fbfb693135818f95a81d461b54d2d3a359ab1e,Much evidence supports the notion that angiote...,Research on the renin-angiotensin system (RAS)...
2,ce40a0a061eaf660c43bf64cc6034b737573e901,Type I interferons (IFN-a/h) are potent antivi...,Viral infections in children and young adults ...
3,6b06e85fdfada57c51d4a475cfac50c21a80289f,,"Since it was first reported in Wuhan, China, i..."
4,d83c529b63855bf62a7a302bfb11d5eb950aa6bc,Feline infectious peritonitis virus (FIPV) cau...,"Feline coronaviruses (FCoVs), which belong to ..."


In [9]:
df_covid

,paper_id,abss,body_text
0,b5d797140d4b3daf7b7ea753783daaad476e2eb0,Sequence characterization of the near full-len...,"T he sequences of two HIV-1 isolates, BCF-Diou..."
1,27fbfb693135818f95a81d461b54d2d3a359ab1e,Much evidence supports the notion that angiote...,Research on the renin-angiotensin system (RAS)...
2,ce40a0a061eaf660c43bf64cc6034b737573e901,Type I interferons (IFN-a/h) are potent antivi...,Viral infections in children and young adults ...
3,6b06e85fdfada57c51d4a475cfac50c21a80289f,,"Since it was first reported in Wuhan, China, i..."
4,d83c529b63855bf62a7a302bfb11d5eb950aa6bc,Feline infectious peritonitis virus (FIPV) cau...,"Feline coronaviruses (FCoVs), which belong to ..."
...,...,...,...
24995,0cca2b3547f329ac1ade31d793e0aceae8cb7af5,Background: Kawasaki disease is the leading ca...,"Kawasaki disease is an acute, self-limited, sy..."
24996,1657b88c4e6f3effe026a66456712234e576e6a3,,"incidence (ie, rate of new infections) to meas..."
24997,1ffaef89637069dfb39d3da07bd3ef8ead63e3ae,"Oligo synthesis has a long history, beginning ...",DNA is the predominant information carrier for...
24998,f739a2568e7a58339a22b0cb89ace39f9127d4dd,Yersinia pestis is the causative agent of plag...,Yersinia pestis is the causative agent of plag...


In [12]:
h=[]
for i in range(0,50):
    s=create_summary(df_covid['body_text'][i])
    h.append(s)
#h=np.asarray(h)
#df_covid['Summary']=h

In [13]:
s

'They also show sizable regions that suggest some relationship with subtypes A1 and H. 5 A more detailed phylogenetic evaluation of the subtype composition of the BCF-Kita genome was made by splitting the genome alignment into smaller segments of variable size between visible breakpoints, as described above. The major portion of the env gene is mostly like subtype H, which would support the original observation placing this strain into this group by HMA analysis. Our previous sequence analysis of the protease and partial reverse tran-scriptase (RT) genes (1190 bases) showed that these strains contain areas suggesting F=G and A=G recombination, respectively. Bootscan plots showed similar crossover points (data not shown)\nOur data indicate that these two NED panel viruses from the DRC are complex recombinants, consistent with previous It is highly likely that BCF-Kita is related to another reported isolate FR.BCB79, also from Hô pital Bichat-Claude Bernard, 16 potentially from the same 

In [14]:
df_covid['body_text'][0]

"T he sequences of two HIV-1 isolates, BCF-Dioum (Dioum) and BCF-Kita (Kita), contributed by F. Brun-Vezinet (Hô pital Bichat-Claude Bernard, France) to the NED (NIH-European Network for Virology Assurance-DoD, U.S. Military HIV Program) panel, were characterized further. This panel was developed in 1996 to serve as a reference panel of well-characterized but minimally passaged HIV-1 strains, with diverse geographic origins 1 and sequence diversity. The isolates were obtained, for the most part, prior to widespread use of antiretroviral therapy and should therefore be considered to be sensitive to antiretroviral therapies. The BCF-Kita isolate was from an asymptomatic individual living in France who was originally from the Democratic Republic of Congo (DRC) and was believed to be infected in the DRC before arrival in France. BCF-Dioum was isolated in France during a primary infection from a person whose HIV-positive partner was from the DRC, and believed to be infected in the DRC befor

In [ ]:
df_covid.to_json(r'/home/harsh/CDAC/CORD-19-research-challenge/Covid_Data.json')